In [1]:
!pip install pylsd
!pip install 'ocrd-fork-pylsd == 0.0.3'

     |████████████████████████████████| 44 kB 519 kB/s eta 0:00:011
  Created wheel for pylsd: filename=pylsd-0.0.2-py3-none-any.whl size=46364 sha256=4ff2fff4abb340b93849fc394cd6026cba0935b860355ec85be6a4ddae8ca318
  Stored in directory: /Users/nakamurasatoru/Library/Caches/pip/wheels/9b/63/9d/790450d057de8c5fc9ed261526ce13048db7e365ef5f53ca0e
Successfully built pylsd
     |████████████████████████████████| 47 kB 514 kB/s eta 0:00:011


In [9]:
!pip install gdown

In [12]:
!gdown "https://drive.google.com/uc?id=1lu739RJi0wNQ5neWJ71zzaDVZqhvjoE5"

Downloading...
From: https://drive.google.com/uc?id=1lu739RJi0wNQ5neWJ71zzaDVZqhvjoE5
To: /Users/nakamurasatoru/git/d_nijl/ruler/best.pt
100%|█████████████████████████████████████████| 173M/173M [10:28<00:00, 276kB/s]


In [28]:
key = "kyoto"

####

collection = {
    "sat": {
        "url" : "https://candra.dhii.jp/iipsrv/iipsrv.fcgi?IIIF=/kakouzou_pub/001_1/0001s.tif/full/full/0/default.jpg"
    },
    "nijl": {
        "url" : "https://kotenseki.nijl.ac.jp/api/iiif/100302677/v4/KANS/KANS-00281/KANS-00281-00002.tif/full/full/0/default.jpg"
    },
    "kyoto": {
        "iiif" : "https://rmda.kulib.kyoto-u.ac.jp/iiif/RB00020027/RB00020027_00001_0.ptif/16512,327,1872,1200/full/0/default.jpg"
    },
    "kyushu": {
        "url" : "https://catalog.lib.kyushu-u.ac.jp/image/iiif/820/1467768/467234.tiff/full/full/0/default.jpg"
    }
}

#####

item = collection[key]

if "iiif" in item:
  url = item["iiif"]
  type = "iiif"
else:
  url = item["url"]
  type = "url"
    
!python load.py $url $type

入力画像サイズ: 1872 px x 1200 px


In [31]:
!python main.py $url -t $type -s True

Using cache found in /Users/nakamurasatoru/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-17 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
<IPython.core.display.Image object>
<IPython.core.display.Image object>
<IPython.core.display.Image object>
<IPython.core.display.Image object>
<IPython.core.display.Image object>
1mmあたりのピクセル数: 26 pixes
検出した定規画像の実サイズ: 193 mm x 8 mm
入力画像の実サイズ: 199 mm x 132 mm
（追加）フル画像の実サイズ: 2026 mm x 2827 mm
